In [1]:
# Importing the libraries
import pandas as pd

In [2]:
data_1st_night_time = pd.read_csv('data/1st night time.csv')
data_midnight = pd.read_csv('data/midnight.csv')
data_saturday_aft_1 = pd.read_csv('data/saturday aft 1.csv')


data = pd.concat([data_1st_night_time, data_midnight, data_saturday_aft_1], ignore_index=True)
data.to_csv('data/data.csv', index=False)

In [3]:
data.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,103.12.198.162,39.62.1.38,UDP,1274,443 > 45841 Len=1232
1,2,-0.005975,39.62.1.38,142.250.181.54,TCP,78,2655 > 443 [ACK] Seq=1 Ack=1 Win=3708 Len=0 ...
2,3,-0.005898,39.62.1.38,103.12.198.146,TCP,66,44538 > 443 [ACK] Seq=1 Ack=1 Win=1736 Len=0...
3,4,-0.005898,39.62.1.38,103.12.198.146,TCP,66,44540 > 443 [ACK] Seq=1 Ack=1 Win=1734 Len=0...
4,5,-0.005889,39.62.1.38,103.12.198.146,TCP,66,44540 > 443 [ACK] Seq=1 Ack=2761 Win=1736 Le...


In [4]:
data.describe()

,No.,Time,Length
count,2.327257e+06,2.327257e+06,2.327257e+06
mean,5.375444e+05,4.174914e+01,8.861782e+02
std,3.314515e+05,2.477640e+01,7.713895e+02
min,1.000000e+00,-5.975000e-03,2.400000e+01
25%,2.437680e+05,2.059823e+01,8.200000e+01
50%,5.346750e+05,3.997793e+01,1.282000e+03
75%,8.255820e+05,6.407431e+01,1.462000e+03
max,1.119795e+06,8.639050e+01,2.123800e+04


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327257 entries, 0 to 2327256
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   No.          int64  
 1   Time         float64
 2   Source       object 
 3   Destination  object 
 4   Protocol     object 
 5   Length       int64  
 6   Info         object 
dtypes: float64(1), int64(2), object(4)
memory usage: 124.3+ MB


In [6]:
# 1. Eksik Değerlerin İncelenmesi ve Temizlenmesi
print("Eksik değerler:")
print(data.isnull().sum())

# Eksik veriler varsa bunları doldurabiliriz (örneğin, forward fill yöntemiyle) veya tamamen çıkarabiliriz.
data.dropna(inplace=True)

Eksik değerler:
No.               0
Time              0
Source            0
Destination       0
Protocol          0
Length            0
Info           2247
dtype: int64


In [7]:
;# 2. Zaman Formatının Düzeltilmesi
# Zaman sütununun negatif değerlerini düzeltmek
data['Time'] = data['Time'].abs()

In [8]:
# 3. Gereksiz Sütunların Çıkarılması
# No. sütunu analize katkı sağlamadığı için çıkarılabilir
data.drop(columns=['No.'], inplace=True)

In [9]:
# 4. Geçerli IP Adreslerini Kontrol Etmek
def is_valid_ip(ip):
    parts = ip.split('.')
    return len(parts) == 4 and all(p.isdigit() and 0 <= int(p) <= 255 for p in parts)

data = data[data['Source'].apply(is_valid_ip) & data['Destination'].apply(is_valid_ip)]

In [10]:
# 5. Verilerin Vektörleştirilmesi
data['Source_IP1'], data['Source_IP2'], data['Source_IP3'], data['Source_IP4'] = zip(*data['Source'].apply(lambda x: x.split('.')))
data['Destination_IP1'], data['Destination_IP2'], data['Destination_IP3'], data['Destination_IP4'] = zip(*data['Destination'].apply(lambda x: x.split('.')))


In [11]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np

# Protokol sütunu için one-hot encoding
encoder = OneHotEncoder(sparse_output=False)  # 'sparse' yerine 'sparse_output' kullanın
protocol_encoded = encoder.fit_transform(data[['Protocol']])

# IP adreslerini sayısal değerlere dönüştürme
data['Source_IP'] = data['Source_IP1'].astype(int) * 256**3 + data['Source_IP2'].astype(int) * 256**2 + data['Source_IP3'].astype(int) * 256 + data['Source_IP4'].astype(int)
data['Destination_IP'] = data['Destination_IP1'].astype(int) * 256**3 + data['Destination_IP2'].astype(int) * 256**2 + data['Destination_IP3'].astype(int) * 256 + data['Destination_IP4'].astype(int)

# Sonuçta protokol ve IP adresi verilerini birleştirme
final_vectors = np.hstack([data[['Source_IP', 'Destination_IP']].values, protocol_encoded])

# Veriyi normalleştirme
scaler = MinMaxScaler()
data[['Time', 'Length']] = scaler.fit_transform(data[['Time', 'Length']])

# İşlenmiş veriyi kaydetme
data.to_csv('data/processed_data.csv', index=False)

Makine Öğrenmesi Modeli Oluşturma

In [12]:
import faiss

# Vektörlerinizi oluşturun
vectors = np.array(final_vectors, dtype='float32')

# FAISS index oluşturun
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)


ImportError: DLL load failed while importing _swigfaiss: Belirtilen modül bulunamadı.

In [15]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Modeli ve tokenizer'ı yükle
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


ImportError: DLL load failed while importing _C: Belirtilen modül bulunamadı.

In [16]:
inputs = tokenizer.encode('Soru: [Kullanıcı sorgusu]', return_tensors='pt')
outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)


NameError: name 'tokenizer' is not defined

In [17]:
user_query = "Kullanıcı sorgusu burada"
# Bilgi alma kısmı
retrieved_logs = search_logs(user_query)  # search_logs fonksiyonu sizin oluşturduğunuz fonksiyon olmalı
# Yanıt oluşturma
answer = generate_answer(user_query, retrieved_logs)  # generate_answer fonksiyonu sizin oluşturduğunuz fonksiyon olmalı


NameError: name 'search_logs' is not defined